In [23]:
import importlib
import REBRNN
import torch
import pickle
importlib.reload(REBRNN)

<module 'REBRNN' from '/Users/tiril/Documents/IndividualProject/nuclear_repo/knowledge_extraction/relation_extraction/REBRNN.py'>

In [24]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model for NER 
model_name = 'dslim/distilbert-NER'

# Custom keywords
try:
    with open('keyword_matching/directory.pkl', 'rb') as file:
        keywords = pickle.load(file)
except FileNotFoundError or FileExistsError:
    with open('directory.pkl', 'rb') as file:
        keywords = pickle.load(file)

# Hyperparameters
input_size = 512
num_layers = 4          # may require tuning
hidden_size = 256       # may require tuning
num_classes = 97        # 96 different relations plus '0' for no relation
learning_rate = 0.001   # may require tuning
batch_size = 64
num_epochs = 5

In [25]:
model = REBRNN.RelationExtractorBRNN(
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    num_classes=num_classes,
    batch_size=batch_size,
    model_name=model_name,
    custom_keywords=keywords,
    threshold=0.7
)

In [19]:
train_raw, _, test_raw, validation_raw = model.load_data(get_distant=False)

length = 100
train = model.preprocessor(train_raw, length=2*length)
test = model.preprocessor(test_raw, length=length)
validation = model.preprocessor(validation_raw, length=length)

Preprocessing took 0.64 minutes. 58 instances (29.00%) exceeded max length.
Preprocessing took 0.33 minutes. 36 instances (36.00%) exceeded max length.
Preprocessing took 0.34 minutes. 33 instances (33.00%) exceeded max length.


In [27]:
model.set_data_loaders(train, validation, test)
